In [1]:
import pandas as pd

# Load the processed CSV file
csv_file_path = "processed_cricket_data.csv"  # Ensure this path matches the file location
crick_df = pd.read_csv(csv_file_path)

# Use crick_df for creating visualizations
print(crick_df.head())  # Optional: Check the loaded data


   Unnamed: 0.1  Unnamed: 0        date          venue match_category team_1  \
0             0           3  2011-02-21      Ahmedabad   League-Match    AUS   
1             1          48  2011-04-02       Wankhede          Final     SL   
2             2          28  2011-03-12         Nagpur   League-Match    IND   
3             3          40  2011-03-20   Eden Gardens   League-Match    ZIM   
4             6          42  2011-03-23         Mirpur          Final     WI   

  team_2  team_1_runs  team_1_wickets  team_2_runs  ...  match_status  \
0    ZIM        262.0             6.0        171.0  ...        played   
1    IND        274.0             6.0        277.0  ...        played   
2     SA        296.0             0.0        300.0  ...        played   
3  KENYA        308.0             6.0        147.0  ...        played   
4    PAK        112.0             0.0        113.0  ...        played   

    winning_team  best_bowler_1 best_bowler_1_wick    best_bowler_2  \
0     Aus

In [2]:
#Load the Data

csv_file_path = "./processed_cricket_data.csv"
crick_df = pd.read_csv(csv_file_path)

In [3]:
#Define Dashboard Layout and Components

from jupyter_dash import JupyterDash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Load the processed CSV file
data_path = "processed_cricket_data.csv"  # Ensure this file is in the same directory
crick_df = pd.read_csv(data_path)

# Initialize the Dash app
app = JupyterDash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("ICC Men’s Cricket World Cup Dashboard", style={"textAlign": "center"}),

    # Dropdown for team selection
    html.Div([
        html.Label("Select a Team:", style={"fontWeight": "bold"}),
        dcc.Dropdown(
            id="team-dropdown",
            options=[{"label": team, "value": team} for team in crick_df["team_1"].unique()],
            value=crick_df["team_1"].unique()[0],  # Default value
            placeholder="Select a team"
        )
    ], style={"width": "50%", "margin": "auto"}),

    # Graph for matches played by year
    html.Div([
        dcc.Graph(id="matches-by-year")
    ]),

    # Graph for team performance over the years
    html.Div([
        dcc.Graph(id="team-performance")
    ]),

    # Graph for Player of the Match analysis
    html.Div([
        dcc.Graph(id="pom-analysis")
    ])
])


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [4]:
#Add Callbacks for Interactivity

from dash.dependencies import Input, Output

# Callback to update matches played by year graph
@app.callback(
    Output("matches-by-year", "figure"),
    Input("team-dropdown", "value")
)
def update_matches_by_year(selected_team):
    # Group data by year
    year_data = crick_df.groupby("world_cup_year").size().reset_index(name="match_count")
    # Create a bar chart
    fig = px.bar(
        year_data,
        x="world_cup_year",
        y="match_count",
        title="Number of Matches Played Each Year",
        labels={"world_cup_year": "Year", "match_count": "Matches Played"}
    )
    return fig

# Callback to update team performance graph
@app.callback(
    Output("team-performance", "figure"),
    Input("team-dropdown", "value")
)
def update_team_performance(selected_team):
    # Filter data for the selected team
    team_data = crick_df[(crick_df["team_1"] == selected_team) | (crick_df["team_2"] == selected_team)]
    team_data = team_data.groupby("world_cup_year").size().reset_index(name="matches")
    # Create a line chart
    fig = px.line(
        team_data,
        x="world_cup_year",
        y="matches",
        title=f"{selected_team} Performance Over the Years",
        labels={"world_cup_year": "Year", "matches": "Matches Played"}
    )
    return fig

# Callback to update Player of the Match analysis
@app.callback(
    Output("pom-analysis", "figure"),
    Input("team-dropdown", "value")
)
def update_pom_analysis(selected_team):
    # Filter data to include matches involving the selected team
    pom_data = crick_df[(crick_df["team_1"] == selected_team) | (crick_df["team_2"] == selected_team)]
    pom_data = pom_data["pom"].value_counts().head(10).reset_index()
    pom_data.columns = ["Player", "Awards"]
    # Create a pie chart
    fig = px.pie(
        pom_data,
        names="Player",
        values="Awards",
        title=f"Top 10 Player of the Match Winners for {selected_team}"
    )
    return fig



In [5]:
#Run the dashboard
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
